In [ ]:
import glob
import pymysql
import pandas as pd
from sqlalchemy import create_engine

from django.conf import settings

In [ ]:
dbc = settings.LEGACY_DB_CONNECTION

In [ ]:
db_connection_str = f"mysql+pymysql://{dbc['USER']}:{dbc['PASSWORD']}@{dbc['HOST']}/{dbc['NAME']}"
db_connection = create_engine(db_connection_str)

In [ ]:
from appcreator.import_utils import run_import
from appcreator.import_utils import delete_all

In [ ]:
file_class_map_dict = {'Stelle': 'stelle'}

In [ ]:
run_import('archiv', file_class_map_dict=None)

In [ ]:
# run_import('archiv', file_class_map_dict=file_class_map_dict)

In [ ]:
# delete_all('archiv')